In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho no [Instagram da Alura](https://www.instagram.com/aluraonline/) e no [LinkedIn da Alura](https://www.linkedin.com/company/alura/) para anúncios.
*   **Verificar a página de artigos da Alura:** Artigos sobre IA e outras tecnologias podem indicar a proximidade de um evento como a Imersão.
*   **Consultar o site da Alura:** Verifique regularmente o site da Alura.

Essas são as melhores maneiras de se manter atualizado sobre os próximos eventos e cursos da Alura.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
config={"tools": [{"google_search":{}}]})

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.

A imersão é um curso 100% online e gratuito, voltado para profissionais de todas as áreas, mesmo que não possuam conhecimento prévio em IA. O curso ensinará como utilizar o Google Gemini para criar projetos reais e aplicáveis ao dia a dia. Ao final do curso, os participantes receberão um certificado da Alura e do Google. Os melhores projetos desenvolvidos durante a imersão serão premiados.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini da Alura', 'Alura Imersão IA Google Gemini']
Páginas utilizadas na resposta: tecmundo.com.br, thallesbenicio.com.br, youtube.com, alura.com.br, rodrigostoledo.com, starten.tech



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A Sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
        )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais.
        Com base na lista de lançamentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o (google_search) para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles, com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("----------------------------------------------------------------")

  plano_do_post = agente_planejador(topico, lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planejador) ---\n")
  display(to_markdown(plano_do_post))
  print("----------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico, plano_do_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("----------------------------------------------------------------")

  post_final = agente_revisor(topico, rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("----------------------------------------------------------------")







🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha! Vamos então criar o post sobre novidades em Agentes de IA

--- Resultado do Agente 1 (Buscador) ---



> Aqui estão algumas notícias recentes sobre lançamentos de agentes de IA, com foco em lançamentos relevantes no último mês:
> 
> 
> Com base nas notícias recentes, aqui estão alguns lançamentos relevantes de agentes de IA, com foco nos últimos meses:
> 
> 1.  **Agentforce da Salesforce:** A Salesforce está focando em agentes de IA com o Agentforce, uma nova camada em sua plataforma que permite que as empresas criem e implementem agentes de IA que podem agir de forma autônoma em qualquer função de negócios. Esses agentes vão além de chatbots e copilotos, usando habilidades avançadas de raciocínio para tomar decisões e agir, como resolver casos de clientes, qualificar leads de vendas e otimizar campanhas de marketing.
> 2.  **Atualizações da IBM com o watsonx Orchestrate:** A IBM anunciou atualizações em seu watsonx Orchestrate, oferecendo soluções inovadoras para integrar, personalizar e implementar agentes, além de disponibilizar agentes pré-construídos para áreas como RH, vendas e compras. Esses agentes contam com integração com mais de 80 aplicações empresariais, facilitando a orquestração de projetos complexos.
> 3.  **Atualizações do Anthropic no agente Claude:** O agente Claude agora pode determinar se uma pesquisa online pode aprimorar a resposta fornecida, garantindo informações atualizadas e com referências diretas às fontes originais.
> 4.  **Agent.ai:** É uma plataforma que se posiciona como uma rede profissional para agentes de inteligência artificial. A plataforma oferece uma variedade de agentes de IA projetados para atender a diversas necessidades, desde pesquisa de mercado até otimização da conversão de websites.
> 5.  **Novas Ferramentas e Frameworks para Desenvolvedores:** Ferramentas como Microsoft AutoGen, Phidata, PromptFlow e OpenAI Swarm estão sendo desenvolvidas para oferecer novas ferramentas e capacidades para desenvolvedores e pesquisadores.


----------------------------------------------------------------

--- Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos de agentes de IA que você forneceu, vou criar um plano de conteúdo para cada um, identificar o tema mais relevante e, em seguida, elaborar um plano de postagem detalhado para esse tema.
> 
> **1. Agentforce da Salesforce**
> 
> *   **Pontos Relevantes:**
>     *   Capacidade de criar agentes de IA que atuam autonomamente em diversas funções de negócios.
>     *   Vai além de chatbots, tomando decisões e agindo em tarefas como resolução de casos, qualificação de leads e otimização de campanhas.
>     *   Foco em habilidades avançadas de raciocínio para a tomada de decisão.
> *   **Possíveis Títulos:**
>     *   "Salesforce Agentforce: A revolução dos agentes de IA autônomos nos negócios"
>     *   "Como o Agentforce da Salesforce está transformando a automação empresarial com IA"
> 
> **2. Atualizações da IBM com o watsonx Orchestrate**
> 
> *   **Pontos Relevantes:**
>     *   Integração, personalização e implementação de agentes de IA.
>     *   Agentes pré-construídos para RH, vendas e compras.
>     *   Integração com mais de 80 aplicações empresariais.
> *   **Possíveis Títulos:**
>     *   "IBM watsonx Orchestrate: A solução completa para orquestrar agentes de IA em sua empresa"
>     *   "Agentes de IA pré-construídos da IBM: Acelere a automação em RH, vendas e compras"
> 
> **3. Atualizações do Anthropic no agente Claude**
> 
> *   **Pontos Relevantes:**
>     *   Capacidade de determinar se uma pesquisa online pode aprimorar a resposta.
>     *   Informações atualizadas e com referências diretas às fontes originais.
> *   **Possíveis Títulos:**
>     *   "Anthropic Claude: Agente de IA agora com pesquisa integrada para respostas mais precisas"
>     *   "A evolução do Claude: Como a Anthropic está tornando seu agente de IA ainda mais inteligente"
> 
> **4. Agent.ai**
> 
> *   **Pontos Relevantes:**
>     *   Plataforma como uma rede profissional para agentes de IA.
>     *   Variedade de agentes para pesquisa de mercado, otimização de conversão de websites, entre outros.
> *   **Possíveis Títulos:**
>     *   "Agent.ai: A rede profissional para encontrar o agente de IA ideal para o seu negócio"
>     *   "Descubra a Agent.ai: A plataforma que conecta você aos melhores agentes de IA do mercado"
> 
> **5. Novas Ferramentas e Frameworks para Desenvolvedores**
> 
> *   **Pontos Relevantes:**
>     *   Ferramentas como Microsoft AutoGen, Phidata, PromptFlow e OpenAI Swarm.
>     *   Novas capacidades para desenvolvedores e pesquisadores.
> *   **Possíveis Títulos:**
>     *   "Microsoft AutoGen, Phidata, PromptFlow, OpenAI Swarm: As ferramentas que todo desenvolvedor de IA precisa conhecer"
>     *   "O futuro do desenvolvimento de IA: Novas ferramentas e frameworks para criar agentes incríveis"
> 
> Para identificar o tema mais relevante, vou pesquisar o interesse do público e a discussão em torno de cada um desses lançamentos.
> 
> 
> Com base na minha pesquisa, o tema mais relevante parece ser **Novas Ferramentas e Frameworks para Desenvolvedores**, especificamente o Microsoft AutoGen, Phidata, PromptFlow e OpenAI Swarm. Há um interesse significativo em ferramentas que auxiliem desenvolvedores a criar e orquestrar agentes de IA.
> 
> **Tema:** Novas Ferramentas e Frameworks para Desenvolvedores de Agentes de IA (com foco em AutoGen, Phidata, PromptFlow e OpenAI Swarm)
> 
> **Pontos Relevantes:**
> 
> *   **Democratização do desenvolvimento de IA:** Essas ferramentas estão tornando mais fácil para desenvolvedores (mesmo aqueles sem profundo conhecimento em IA) criar agentes inteligentes.
> *   **Multi-agentes:** AutoGen, Phidata e OpenAI Swarm focam na criação de sistemas onde múltiplos agentes podem interagir para resolver tarefas complexas.
> *   **Aplicações:** Essas ferramentas têm aplicações em diversos setores, desde finanças até atendimento ao cliente.
> *   **Open Source:** Muitas dessas ferramentas são open source, incentivando a colaboração e a inovação.
> 
> **Plano de Postagem Detalhado:**
> 
> *   **Título:** "Desbloqueando o Poder dos Agentes de IA: Um Guia para Desenvolvedores com AutoGen, Phidata, PromptFlow e OpenAI Swarm"
> *   **Introdução:**
>     *   Explicar o que são agentes de IA e por que eles são importantes.
>     *   Breve visão geral do cenário atual de ferramentas e frameworks para desenvolvedores de agentes.
>     *   Apresentar AutoGen, Phidata, PromptFlow e OpenAI Swarm como ferramentas-chave.
> *   **Microsoft AutoGen:**
>     *   O que é AutoGen e como ele simplifica a criação de sistemas multi-agentes.
>     *   Casos de uso: automação de tarefas, resolução de problemas complexos.
>     *   Como começar a usar AutoGen.
> *   **Phidata:**
>     *   O que é Phidata e seu foco em agentes multimodais.
>     *   Recursos principais: Multi-Agent orchestration, Agentic RAG built-in, Structured outputs.
>     *   Exemplo de código simples para criar um agente com Phidata.
> *   **PromptFlow:**
>     *   O que é PromptFlow e como ele otimiza o ciclo de desenvolvimento de aplicações de IA.
>     *   Integração com Azure AI Foundry e Azure Machine Learning studio.
>     *   Casos de uso: Prototipagem, experimentação e implantação de aplicações de IA.
> *   **OpenAI Swarm:**
>     *   O que é OpenAI Swarm e como ele facilita a coordenação de múltiplos agentes.
>     *   Conceitos chave: Agentes e handoffs.
>     *   Como Swarm se compara a outras ferramentas como LangGraph.
> *   **Comparação:**
>     *   Uma tabela comparando os recursos, vantagens e desvantagens de cada ferramenta.
>     *   Qual ferramenta é mais adequada para qual tipo de projeto.
> *   **Conclusão:**
>     *   Reafirmar a importância dessas ferramentas para o futuro do desenvolvimento de IA.
>     *   Incentivar os desenvolvedores a explorar e experimentar com essas ferramentas.
>     *   Chamada para ação: compartilhar suas experiências e projetos criados com essas ferramentas.
> 
> Este plano deve fornecer um guia completo para desenvolvedores interessados em explorar o mundo dos agentes de IA, capacitando-os a criar soluções inovadoras e automatizadas.
> 


----------------------------------------------------------------

--- Resultado do Agente 3 (Redator) ---



> 🚀 Dando superpoderes aos devs de IA! 🚀
> 
> Já ouviu falar nos Agentes de IA? 🤖 Eles são a nova fronteira da automação e a Alura te apresenta as ferramentas que vão te colocar na linha de frente dessa revolução!
> 
> 🔥 AutoGen (Microsoft): Crie sistemas complexos com múltiplos agentes interagindo! Imagine a sinergia!
> 🛠️ Phidata: Agentes multimodais ao seu alcance! Orquestração multi-agente e RAG integrados para turbinar seus projetos.
> ✨ PromptFlow: Otimize o desenvolvimento de apps de IA como nunca! Experimente, prototipe e implemente com Azure AI Foundry e Azure Machine Learning Studio.
> 🤝 OpenAI Swarm: Coordene vários agentes como um maestro! Orquestre tarefas complexas com facilidade.
> 
> E aí, qual dessas ferramentas te deixou mais curioso(a)? 🤔 Conta pra gente nos comentários e bora dominar o futuro da IA! 🚀
> 
> #Alura #InteligenciaArtificial #Desenvolvimento #Inovacao
> 


----------------------------------------------------------------

--- Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda melhor para engajar o público jovem do Instagram. Aqui estão algumas sugestões:
> 
> *   **Emojis:** Usar mais emojis pode deixar o post mais dinâmico e atraente para o público do Instagram.
> *   **Linguagem:** Tornar a linguagem um pouco mais casual e direta pode ajudar a criar uma conexão maior com os seguidores.
> *   **Chamada para Ação (CTA):** A CTA pode ser mais direta e focada em gerar interação imediata.
> *    **Hashtags:** Adicionar hashtags mais específicas pode aumentar a visibilidade do post.
> 
> **Rascunho revisado:**
> 
> 🚀 DEU A LOUCA NA IA: AGENTES CHEGARAM PRA DAR SUPERPODERES AOS DEVS! 🚀
> 
> Já ouviram falar dos Agentes de IA? 🤖 Eles são a nova onda da automação e a Alura tá te mostrando as ferramentas pra você SURFAR nessa revolução! 🌊
> 
> 🔥 **AutoGen (Microsoft):** Quer criar sistemas complexos? Com ele, vários agentes trabalham JUNTOS! 🤝
> 🛠️ **Phidata:** Agentes que entendem TUDO! 🤯 Orquestração e RAG integrados pra TURBINAR seus projetos.
> ✨ **PromptFlow:** Desenvolva apps de IA de forma FÁCIL e RÁPIDA! 💨 Teste, crie e implemente com Azure AI.
> 🤝 **OpenAI Swarm:** Controle VÁRIOS agentes como um maestro! 🎼 Tarefas complexas? Moleza!
> 
> E aí, qual dessas ferramentas te deixou mais CURIOSO(A)? 🤔 Conta aqui nos comentários e vamo DOMINAR o futuro da IA! 🚀
> 
> \#Alura \#InteligenciaArtificial \#AgentesDeIA \#InovacaoTecnologica \#DesenvolvimentoDeSoftware \#Automacao
> 
> 


----------------------------------------------------------------
